<a href="https://colab.research.google.com/github/tosittig/CASAIS/blob/main/W2_4_3_Pretrained_Summarization_4colab_OUTPUT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Summarization
In this notebook, we try different text summarization models. We will use the cnn daily mail dataset.

In [ ]:
# Run the following command if you have not installed the package `datasets` before:
!pip install datasets

# If you get an error in the cells below (NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported.)
# Run the line below - there seems to be an issue with fsspec 2023.10.0:
!pip install fsspec==2023.9.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.6 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

Here we load the dataset:

In [ ]:
dataset = load_dataset('cnn_dailymail', '3.0.0', download_mode='force_redownload', verification_mode='no_checks')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
print(f"Features: {dataset['train'].column_names}")

Features: ['article', 'highlights', 'id']


As an example we will use the following (randomly selected) text:

In [ ]:
sample = dataset["train"][1]
print(f"""
Article (excerpt of 500 characters, total length: {len(sample["article"])}):
""")
print(sample["article"][:500])
print(f'\nSummary (length: {len(sample["highlights"])}):')
print(sample["highlights"])


Article (excerpt of 500 characters, total length: 4051):

Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most s

Summary (length: 281):
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .


We will collect the generated summaries of each model in a dictionary:

In [ ]:
sample_text = dataset["train"][1]["article"][:2000]
summaries = {}

### Sentence Tokenization
In summarization, it's common practice to separate the summary sentences by a newline character. A simple approach replacing a full stop, however, would fail for strings like "U.N." or "U.S.". Within the Natural Language Toolkit (NLTK), the function `sent_tokenize` uses more advanced algorithms to differentiate between the end of a sentence and punctuation that occurs within an abbreviation.

In [ ]:
!pip install nltk

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Here's an example of how the `sent_tokenize` works:

In [ ]:
string = "The U.S. are a country. The U.N. is an organization."
sent_tokenize(string)

['The U.S. are a country.', 'The U.N. is an organization.']

## Testing out Different Summarization approaches:

### Summarization Baseline
A simplistic baseline is to just use the first 3 sentences of each article. This approach can easily be implemented using the function `sent_tokenize`:

In [ ]:
def three_sentence_summary(text):
    return "\n".join(sent_tokenize(text)[:3])

In [ ]:
summaries["baseline"] = three_sentence_summary(sample_text)

### GPT-2
To use version 2 of GPT for text summarization, we engineer a prompt by adding "TL; DR" to the end of the input text. "TL; DR" stands for "too long; don't read" and is often used e.g., in Reddit posts to indicate the short version of a longer post. We thus direct GPT into the direction of a short version of the input text.

In [ ]:
from transformers import pipeline, set_seed

set_seed(42)
pipe = pipeline("text-generation", model="gpt2")
gpt2_query = sample_text + "\nTL;DR:\n"
pipe_out = pipe(gpt2_query, max_length=512, clean_up_tokenization_spaces=True)
summaries["gpt2"] = "\n".join(
    sent_tokenize(pipe_out[0]["generated_text"][len(gpt2_query) :]))

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


### T5
T5, the Text-to-Text Transfer Transformer, is trained using a universal approach to formulate all tasks as text-to-text tasks. T5 was trained on both unsupervised (to reconstruct the masked words) and supervised data, including summarization. Using the same prompts as in the pretraining, we can thus directly do summarization. The input format to summarize a document is `summarize: <ARTICLE>'` (Translation, by the way, can be prompted via `translate English to German: <TEXT>'`).
We write a small function to prepend `summarize` to our texts:

In [ ]:
pipe = pipeline("summarization", model="t5-small")
pipe_out = pipe(sample_text)
summaries["t5"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
print(summaries["t5"])

inmates with most severe mental illnesses are incarcerated until they're ready to appear in court .
most often, they face drug charges or charges of assaulting an officer .
they end up on the ninth floor severely mentally disturbed, but not getting real help .


### BART
The Bidirectional and Auto-Regressive
Transformers (BART) combine the approach of BERT and autoregressive transfomer as in GPT. A checkpoint specifically finetuned for the given dataset is available from the Huggingface Hub:

In [ ]:
pipe = pipeline("summarization", model="facebook/bart-large-cnn")
pipe_out = pipe(sample_text)
summaries["bart"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

## Comparing Summaries
Now, let's look at the different summaries we have obtained using the different models:

In [ ]:
print("GROUND TRUTH")
print(dataset["train"][1]["highlights"])
print("")

for key, val in summaries.items():
  print('****')
  print(key.upper())
  print(val)

GROUND TRUTH
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .

****
BASELINE
Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.
Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.
MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."
****
GPT2

The prison in jail that serves the mentally ill is a mental health facility with 12,000 inmates.
It was founded by psychiatrist Dr. Daniel M. Dufour, who was born in New Orleans by his parents when he was 12.

In [ ]:
# !pip install rouge_score
# !pip install sacrebleu

### BLUE Metric

In [ ]:
!pip install sacrebleu
from datasets import load_metric

bleu_metric = load_metric("sacrebleu")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 2.7 MB/s eta 0:00:00


<ipython-input-20-52906bfbdb2f>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu_metric = load_metric("sacrebleu")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
import pandas as pd
import numpy as np

bleu_metric.add(
    prediction="the the the the the the",
    reference=["the cat is on the mat"])

results = bleu_metric.compute(smooth_method="floor", smooth_value=0)
results["precisions"] = [np.round(p, 2) for p in results["precisions"]]
pd.DataFrame.from_dict(results, orient="index", columns=["Value"])

,Value
score,0.0
counts,"[2, 0, 0, 0]"
totals,"[6, 5, 4, 3]"
precisions,"[33.33, 0.0, 0.0, 0.0]"
bp,1.0
sys_len,6
ref_len,6


In [ ]:
bleu_metric.add(prediction="the cat is on mat",
                reference=["the cat is on the mat"])
results = bleu_metric.compute(smooth_method="floor", smooth_value=0)
results["precisions"] = [np.round(p, 2) for p in results["precisions"]]
pd.DataFrame.from_dict(results, orient="index", columns=["Value"])

,Value
score,57.893007
counts,"[5, 3, 2, 1]"
totals,"[5, 4, 3, 2]"
precisions,"[100.0, 75.0, 66.67, 50.0]"
bp,0.818731
sys_len,5
ref_len,6


In [ ]:
records = []
reference = dataset["train"][1]["highlights"]
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

for model_name in summaries:
    bleu_metric.add(prediction=summaries[model_name], reference=[reference])
    results = bleu_metric.compute(smooth_method='floor', smooth_value=0)
    results['precisions'] = [np.round(p, 2) for p in results['precisions']]

    bleu_dict = dict(results)
    records.append(bleu_dict)

pd.DataFrame.from_records(records, index=summaries.keys())

,score,counts,totals,precisions,bp,sys_len,ref_len
baseline,8.379861,"[28, 9, 4, 3]","[90, 89, 88, 87]","[31.11, 10.11, 4.55, 3.45]",1.000000,90,57
gpt2,0.000000,"[12, 0, 0, 0]","[71, 70, 69, 68]","[16.9, 0.0, 0.0, 0.0]",1.000000,71,57
t5,0.000000,"[10, 1, 0, 0]","[47, 46, 45, 44]","[21.28, 2.17, 0.0, 0.0]",0.808345,47,57
bart,21.296284,"[25, 13, 10, 6]","[57, 56, 55, 54]","[43.86, 23.21, 18.18, 11.11]",1.000000,57,57


We see that bart is by far the best model - however, this is due to the facxt that both gpt2 and t5 do not have an overlap in the n-grams for n larger than 2. Since the overall score is an exponentially weighted product, the overall score of these two methods is 0.

### ROUGE Metric

In [ ]:
!pip install rouge_score

rouge_metric = load_metric("rouge")

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=408074267b4cd6971912e8f9412b51646ac608d95cd36eb4a2407c43b47363e0
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
reference = dataset["train"][1]["highlights"]
records = []
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

for model_name in summaries:
    rouge_metric.add(prediction=summaries[model_name], reference=reference)
    score = rouge_metric.compute()
    rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
    records.append(rouge_dict)
pd.DataFrame.from_records(records, index=summaries.keys())

,rouge1,rouge2,rougeL,rougeLsum
baseline,0.365079,0.145161,0.206349,0.285714
gpt2,0.252252,0.018349,0.144144,0.234234
t5,0.195652,0.022222,0.108696,0.173913
bart,0.475248,0.222222,0.316832,0.415842


We see that also based on a proper evaluation, the bart model achieves a rouge scores that are quite a bit better than the baseline approach. gpt2 and t5, however, are again quite a bit below in their performance.